In [2]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")

In [8]:
import particlezoo as zoo
import liesym as ls

In [9]:
p = zoo.Field("psi", "1/2", {"U(1)_x": "q"})
p.conjugate_name

conjugate(psi)

In [10]:
s = zoo.Symmetry("U(1)_x", ls.U1(), True, "g", tag="x")
s

Symmetry(U(1)_x)

In [11]:
l = zoo.Lagrangian([p], [s], "test")

In [12]:
l._get_sym("U(1)_x")

Symmetry(U(1)_x)

In [41]:
from particlezoo import Field, Symmetry
from tikzfeynwrap import TikzFeynWrap
from typing import NamedTuple
from sympy import latex

In [42]:
def id_gen():
    import string
    i = -1
    def _get_id():
        nonlocal i
        i += 1
        return "a" + str(i)
    return _get_id


class Node2(NamedTuple):
    name: str
    connectedTo: list[tuple]
    key: str

class Diagram(NamedTuple):
    heads: list[Node2]
    horizontal: tuple

allowed = [
            "charged scalar",
            "scalar",
            "majorana",
            "anti majorana",
            "gluon",
            "photon",
            "ghost",
            "charged boson",
            "fermion"
        ]

def _create_vertex(key, name=None):
    if name:
        return f"{key} [particle=\({name}\)]"
    return key

def _create_line(key, line_type=None):
    if line_type:
        if line_type in allowed:
            return f"-- [{line_type}]"
        else:
            raise ValueError(f"Bad tikz-feynman option: {line_type}")
    return f"--"


def fermion_kinetic_energy(field: Field, sym: Symmetry):
    id_ = id_gen()

    fname = latex(field.name)
    fname_conj = latex(field.conjugate_name)
    gauge_boson = latex(sym.gauge_name)


    gbout = Node2(gauge_boson, None, id_())
    conjout = Node2(fname_conj, None, id_())
    interaction = Node2(None,[
        (gbout, 'fermion', id_()),
        (conjout, 'photon', id_())
    ], id_())
    pin = Node2(fname, [(interaction, 'fermion', id_())], id_())
    # return Diagram([pin], (interaction, gbout))
    return pin

ex = fermion_kinetic_energy(p, s)   

print(ex)

total = []

# def parse_path(n: Node2, log: list):
import copy

def parse_heads(head: Node2, l: list):
    vertex = _create_vertex(head.key, head.name)
    l.append(vertex)

    if head.connectedTo is None:
        total.append(copy.deepcopy(l))
        l.clear()
        return

    for node, name, key in head.connectedTo:
        path = _create_line(key, name)
        l.append(path)
        parse_heads(node, copy.deepcopy(l))

l = []
parse_heads(ex, l)
total


Node2(name='\\psi', connectedTo=[(Node2(name=None, connectedTo=[(Node2(name='\\mathtt{\\text{A\\_x}}', connectedTo=None, key='a0'), 'fermion', 'a2'), (Node2(name='\\overline{\\psi}', connectedTo=None, key='a1'), 'photon', 'a3')], key='a4'), 'fermion', 'a5')], key='a6')


[['a6 [particle=\\(\\psi\\)]',
  '-- [fermion]',
  'a4',
  '-- [fermion]',
  'a0 [particle=\\(\\mathtt{\\text{A\\_x}}\\)]'],
 ['a6 [particle=\\(\\psi\\)]',
  '-- [fermion]',
  'a4',
  '-- [fermion]',
  '-- [photon]',
  'a1 [particle=\\(\\overline{\\psi}\\)]']]

In [4]:
# wrapper = TikzFeynWrap().startup()

In [43]:
def dec(x):
    return f"""\\feynmandiagram [horizontal=a6 to a0] {{
        {x}
        }};"""

t = dec(" ".join(total[0]))
print(t)
x = wrapper(t, return_path=True)

\feynmandiagram [horizontal=a6 to a0] {
        a6 [particle=\(\psi\)] -- [fermion] a4 -- [fermion] a0 [particle=\(\mathtt{\text{A\_x}}\)]
        };


In [52]:
a,r = wrapper(r"""
\feynmandiagram [horizontal=a to b] {
        i1 [particle=\(e^{-}\)] -- [fermion] a -- [fermion] i2 [particle=\(e^{+}\)],
        a -- [photon, edge label=\(\gamma\), momentum'=\(k\)] b,
        f1 [particle=\(\mu^{+}\)] -- [fermion] b -- [fermion] f2 [particle=\(\mu^{-}\)],

        };
""", return_path=True, return_output=True)

In [50]:
from IPython.display import SVG, display

display(SVG(a))

In [56]:
print(str(r))

ExecResult(exit_code=0, output=b'This is LuaHBTeX, Version 1.13.2 (TeX Live 2021) \n restricted system commands enabled.\n(./output.tex\nLaTeX2e <2021-06-01> patch level 1\n L3 programming layer <2021-08-27>\n(/usr/local/texlive/2021/texmf-dist/tex/latex/standalone/standalone.cls\nDocument Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan\ndalone\n(/usr/local/texlive/2021/texmf-dist/tex/latex/tools/shellesc.sty)\n(/usr/local/texlive/2021/texmf-dist/tex/generic/iftex/ifluatex.sty\n(/usr/local/texlive/2021/texmf-dist/tex/generic/iftex/iftex.sty))\n(/usr/local/texlive/2021/texmf-dist/tex/latex/xkeyval/xkeyval.sty\n(/usr/local/texlive/2021/texmf-dist/tex/generic/xkeyval/xkeyval.tex\n(/usr/local/texlive/2021/texmf-dist/tex/generic/xkeyval/xkvutils.tex\n(/usr/local/texlive/2021/texmf-dist/tex/generic/xkeyval/keyval.tex))))\n(/usr/local/texlive/2021/texmf-dist/tex/latex/standalone/standalone.cfg)\n(/usr/local/texlive/2021/texmf-dist/tex/latex/base/article.cls\nDocument Cl